In [2]:
import sys
import os
sys.path.append("..")
from globals import ROOT_DIR
import configparser
import csv
import pandas as pd
import numpy as np 
import tweepy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

config = configparser.ConfigParser()
config.read('../config.ini')
consumer_key = config['DEFAULT']['TWITTER_CONSUMER_KEY']
consumer_secret_key = config['DEFAULT']['TWITTER_CONSUMER_SECRET_KEY']
access_token = config['DEFAULT']['TWITTER_ACCESS_TOKEN']
access_token_secret = config['DEFAULT']['TWITTER_ACCESS_TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [3]:
data_file = os.path.join(ROOT_DIR, 'data/founta_data.npy')
data = np.load(data_file)
data = data[()]

In [12]:
path_labels = os.path.join(ROOT_DIR, 'data/founta_data.csv')
labels = pd.read_csv(path_labels, header='infer', index_col=0, squeeze=True).to_dict()

In [13]:
users = [(value['user']['id'], key) for key, value in data.items() if labels[int(key)] == 'hateful']

In [113]:
print(data[users[869][1]]['text'])
users[869]

@TRobinsonNewEra @AmyMek Muslims immigrants are living like they're used to where their from, squalor, filth and violence are all they know.


(613762727, '849061712242368512')

In [4]:
import time 
start = time.time()
timelines = {}
for i in range(6):
    results = np.load(os.path.join(ROOT_DIR, 'data/user_timeline_processed_{}.npz'.format(i)), allow_pickle=True)
    print("Downloading User Timelines, Processed {} / {}".format(i+1, 6))
    results = results['a']
    results = results[()]
    timelines = {**results, **timelines}
print("End time {}".format((time.time() - start) / 60))

End time 21.242652801672616


In [ ]:
list(timelines.keys())[0]

In [5]:
from collections import Counter 

def get_scores(docs):
    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]
    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent)) 
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out
    bigram = gensim.models.Phrases(docs, min_count=5, threshold=100) # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)


    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])
    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(docs, allowed_postags=['NOUN', 'ADJ', 'ADV'])
    data_lemmatized
    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=20, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)
    # Print the Keyword in the 10 topics
#     dominant_keywords = []
#     for i, row_list in enumerate(lda_model[corpus]):
#         row = row_list[0] if lda_model.per_word_topics else row_list            
#         row = sorted(row, key=lambda x: (x[1]), reverse=True)
#         # Get the Dominant topic, Perc Contribution and Keywords for each document
#         for j, (topic_num, prop_topic) in enumerate(row):
#             if j == 0:  # => dominant topic
#                 wp = lda_model.show_topic(topic_num)
#                 dominant_keywords.extend([word for word, prop in wp])
#                 topic_keywords = ", ".join([word for word, prop in wp])
#             else: 
#                 break
#     topic_words = Counter(dominant_keywords).most_common(10)

    #len(doc_lda)
    # Compute Perplexity
    #print('Perplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    #print('Coherence Score: ', coherence_lda)
    # In my experience, topic coherence score, in particular, has been more helpful.
    vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    
    return lda_model.log_perplexity(corpus), coherence_lda, vis, topic_words

In [114]:
user_scores = {}
save_count = 0
for i, (key, value) in enumerate(timelines.items()):
    value = timelines[613762727]
    value = value[:200]
    doc = [tweet for tweet in value]
    if len(doc) == 0:
        continue
    perplexity, coherence, vis, topic_words = get_scores(doc)
    print(topic_words)
    user_scores[key] = (perplexity, coherence)
    if i % 100 == 0:
        print(i)
    if i % 10000 == 0 and i != 0:
        np.savez(os.path.join(ROOT_DIR, 'data/user_lda_scores{}.npz'.format(save_count)), a=user_scores)
        save_count += 1
        user_scores = {}
    break

/Users/ashemagalhaes/miniconda3/envs/mlp/lib/python3.6/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
/Users/ashemagalhaes/miniconda3/envs/mlp/lib/python3.6/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)


[('-PRON-', 94), ('bob', 57), ('not', 56), ('potus', 49), ('when', 49), ('freedomcaucus', 49), ('alley167', 49), ('exhaustartist', 49), ('vote', 49), ('epacheco255', 49)]
0


In [115]:
pyLDAvis.enable_notebook()

vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.196210  0.004889       1        1  16.062380
2      0.230072 -0.150318       2        1  11.768608
19     0.146204  0.277544       3        1   9.348438
16     0.130903 -0.027954       4        1   8.708648
15     0.128472 -0.049645       5        1   7.519624
1      0.046204  0.063858       6        1   4.672150
7      0.009997  0.093247       7        1   4.317151
11    -0.035568  0.019288       8        1   4.189716
0     -0.023535 -0.070839       9        1   4.159512
17    -0.034582  0.011888      10        1   3.732811
18    -0.028116  0.031158      11        1   3.691243
14    -0.076923 -0.019679      12        1   3.205299
13    -0.037801 -0.023917      13        1   3.180196
3     -0.003401 -0.095801      14        1   3.083203
4     -0.060677 -0.004425      15        1   2.632113
6     -0.071954  0.021013      16        1   2.431036
8      0.014671 -0.007370      17        1   2.373343
9     -0.001871 -0.077709      18        1   1.917831
5     -0.073964 -0.015270      19        1   1.613715
12    -0.061921  0.020043      20        1   1.392972, topic_info=     Category       Freq                Term      Total  loglift  logprob
term                                                                     
208   Default  26.000000                 all  26.000000  30.0000  30.0000
232   Default  15.000000                 dem  15.000000  29.0000  29.0000
17    Default  26.000000                 not  26.000000  28.0000  28.0000
11    Default  12.000000           president  12.000000  27.0000  27.0000
12    Default  22.000000                what  22.000000  26.0000  26.0000
351   Default  12.000000             epstein  12.000000  25.0000  25.0000
356   Default  12.000000              acosta  12.000000  24.0000  24.0000
0     Default  31.000000              -PRON-  31.000000  23.0000  23.0000
357   Default  11.000000                 tie  11.000000  22.0000  22.0000
23    Default  14.000000              muslim  14.000000  21.0000  21.0000
13    Default   9.000000              people   9.000000  20.0000  20.0000
110   Default  10.000000               world  10.000000  19.0000  19.0000
27    Default   9.000000     realdonaldtrump   9.000000  18.0000  18.0000
14    Default   6.000000               trump   6.000000  17.0000  17.0000
50    Default   4.000000               obama   4.000000  16.0000  16.0000
111   Default   6.000000            congress   6.000000  15.0000  15.0000
80    Default   9.000000                 how   9.000000  14.0000  14.0000
309   Default   6.000000              census   6.000000  13.0000  13.0000
116   Default   6.000000                baby   6.000000  12.0000  12.0000
158   Default   4.000000                year   4.000000  11.0000  11.0000
59    Default   5.000000                that   5.000000  10.0000  10.0000
149   Default   6.000000             problem   6.000000   9.0000   9.0000
8     Default   7.000000                 law   7.000000   8.0000   8.0000
76    Default   4.000000             nothing   4.000000   7.0000   7.0000
293   Default   2.000000            genflynn   2.000000   6.0000   6.0000
84    Default   5.000000                just   5.000000   5.0000   5.0000
182   Default   3.000000          senschumer   3.000000   4.0000   4.0000
5     Default   4.000000                more   4.000000   3.0000   3.0000
7     Default   3.000000        kamalaharris   3.000000   2.0000   2.0000
135   Default   3.000000               there   3.000000   1.0000   1.0000
...       ...        ...                 ...        ...      ...      ...
280   Topic20   0.019921            separate   1.049152   0.3098  -6.6462
294   Topic20   0.019921               plain   1.049152   0.3098  -6.6462
290   Topic20   0.019921             control   1.049152   0.3098  -6.6462
131   Topic20   0.019921               other   1.067043   0.2928  -6.6462
34    Topic20   0.019921       

In [157]:
sample = torch.zeros((100, 1))
cat = torch.Tensor([[2, 2] for i in range(17)])
cat.shape

torch.Size([17, 2])

In [151]:
sample.shape, cat.shape

(torch.Size([100, 1]), torch.Size([34, 1]))

In [149]:
import torch
res = torch.cat((sample, cat))
res.shape

torch.Size([102, 1])

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
len(users)

In [ ]:
count = 0 
for key, value in data.items():
    if count > 2:
        break
    tweet = value
    count += 1
tweet

In [ ]:
user_id = tweet['user']['id_str']
user_id

In [ ]:
obj = api.user_timeline(user_id)

In [ ]:
obj[0]._json

In [ ]:
status_ids = [item._json['id'] for item in obj]
oldest = status_ids[-1]

In [ ]:
obj3 = api.user_timeline(user_id, count = 3300)

In [ ]:
oldest = obj3[0]._json['id']
oldest

In [ ]:
obj3[-1]._json['created_at']
obj3[-1]._json['id']

In [ ]:
obj4 = api.user_timeline(user_id, count = 3300, since_id = oldest)

In [ ]:
len(obj4)

In [ ]:
obj2[-1]._json

In [ ]:
import time

data_file = os.path.join(ROOT_DIR, 'data/founta_data.npy')
data = np.load(data_file)
data = data[()]

users = [(value['user']['id'], value['id']) for key, value in data.items()]
user_data = {}

# user_data = np.load(os.path.join(ROOT_DIR, 'data/user_data.npy'))
# user_data = user_data[()]
start = time.time()
save_count = 0 
for i, (user_id, tweet_id) in enumerate(users):
    print("On user {}".format(i))
    page_count = 0 
    tweets = []
    for pages in tweepy.Cursor(api.user_timeline, id=user_id, count=200, max_id=tweet_id).pages(): 
        tweets.extend(pages)
        if len(tweets) >= 200:
            break 
            
#     tweets = api.user_timeline(user_id, count = 200, max_id = tweet_id)
    print(len(tweets))
    user_data[user_id] = [tweet._json['text'] for tweet in tweets] #each user has list of text tweets 

    if i % 1000 == 0:
        np.savez(os.path.join(ROOT_DIR, 'data/user_timeline_{}.npz'.format(save_count)), a=user_followers)
        save_count += 1
#     print(time.time() - start)

In [ ]:
print(len(user_data[2892845505]))

In [ ]:
for key, value in user_data.items():
    print(value)
    break

In [ ]:
from collections import defaultdict


#make it blank 
user_data = defaultdict(list)
np.save(os.path.join(ROOT_DIR, 'data/user_data.npy'), user_data)

for user_id in users:
    since_id = None 
    rate_limit = 200
    start_ptr = 0
    end_ptr = start_ptr + rate_limit
    while(True):
        print("Start ptr is at {}".format(start_ptr))

        #load next series of tweets 
        if since_id is not None:
            prev_tweets = tweets 
            if len(tweets) == 0:
                print("Total length of tweets collected: {}".format(len(user_data[user_id])))
                break 
        else:
            tweets = api.user_timeline(user_id, count = rate_limit)
            if len(tweets) == 0:
                print("Total length of tweets collected: {}".format(len(tweets)))
                break 

        since_id = tweets[0]._json['id']

        # load and save results each time 
#         if os.path.exists(os.path.join(ROOT_DIR, 'data/user_data.npy')):
#             user_data = np.load(os.path.join(ROOT_DIR, 'data/user_data.npy'))
#             user_data = user_data[()]
#         else: 
#             user_data = defaultdict(list)
#         print(len(tweets)) 
        for i, tweet in enumerate(tweets):
            tweet = tweet._json
            user_data[user_id].append(tweet['text'])
        np.save(os.path.join(ROOT_DIR, 'data/user_data.npy'), user_data)

        start_ptr += rate_limit 
        end_ptr += rate_limit


In [ ]:
for key, value in user_data.items():
    print(len(value))

In [ ]:
tweets = api.user_timeline('3308335625', count = 3300)
len(tweets)